In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502019


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
5165194,1112266297,N12QNx,0,MASC,39.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,NaN
9353277,1112269758,PmGKP31,0,FEM,19.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN
7553625,1112266386,YNE6rX,0,FEM,34.0,-,-,-,Graduado,Graduado,...,False,False,False,True,False,False,False,False,False,NaN
1909932,1112402512,xkPEobY,1,MASC,26.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.043478
3607340,1112352459,4reQR5M,0,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
14317556,1112353640,ow5pwd4,0,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
10162264,1112280836,JBm0xOk,1,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
2876520,1112420476,wVkWWQL,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,NaN
14670760,1112473287,GNJq0pE,0,MASC,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,NaN
1495506,1112372002,55ADjp,1,FEM,24.0,-,-,-,-,-,...,False,True,False,False,True,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
1561364,1112326333,2zP60rw,0,MASC,60.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,0.000000
9093499,1111818090,MVPXqNL,1,MASC,36.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,False,0.000000
5655219,1112335554,rmRoXl6,0,MASC,33.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.000000
4643445,1112320432,695rA4,0,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,True,False,0.000000
7527833,1110092958,LNKpr9r,1,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
9389654,1112335040,JBeQNPJ,1,MASC,30.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
2131598,1112204682,ZDP4olb,1,MASC,38.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.000000
2459968,1112417700,2zkJ96w,0,MASC,31.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
15796146,1112473363,MVd00lN,0,MASC,39.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
12047360,1112395188,BmBPYAR,0,FEM,24.0,-,-,Graduado,-,-,...,False,True,False,True,False,False,False,False,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281886 Test:  804310


In [28]:
gc.collect()

520

In [29]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=18,n_estimators=1200,colsample_bytree=0.9,n_jobs=-1,random_state=0,silent=False,subsample=1,sumsample_freq=1)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.689857
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.68559
[3]	valid_0's binary_logloss: 0.68124
[4]	valid_0's binary_logloss: 0.67778
[5]	valid_0's binary_logloss: 0.673572
[6]	valid_0's binary_logloss: 0.66991
[7]	valid_0's binary_logloss: 0.665952
[8]	valid_0's binary_logloss: 0.662251
[9]	valid_0's binary_logloss: 0.659438
[10]	valid_0's binary_logloss: 0.656451
[11]	valid_0's binary_logloss: 0.652843
[12]	valid_0's binary_logloss: 0.64921
[13]	valid_0's binary_logloss: 0.64589
[14]	valid_0's binary_logloss: 0.64423
[15]	valid_0's binary_logloss: 0.64162
[16]	valid_0's binary_logloss: 0.638071
[17]	valid_0's binary_logloss: 0.63459
[18]	valid_0's binary_logloss: 0.632071
[19]	valid_0's binary_logloss: 0.62893
[20]	valid_0's binary_logloss: 0.625568
[21]	valid_0's binary_logloss: 0.622272
[22]	valid_0's binary_logloss: 0.61941
[23]	valid_0's binary_logloss: 0.616876
[24]	valid_0's binary_logloss: 0.613784

In [30]:
score=c.score(x_test,y_test)*100
print(score)

83.62497047158433


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV16.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV16.pkl']

In [32]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
27801,67462,1112460307,YD6mxP,MASC,29.0,-,-,-,-,Graduado,...,False,True,False,False,True,True,False,False,False,0.009091
356,66829,1112460141,MV6ZzKo,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.000952
21717,44649,1112426878,60Lv3j,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,0.000000
65544,12057,1111978618,Z2b1o8,FEM,39.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.000000
59125,57622,1112451640,eD8GXr,MASC,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
42471,39707,1112417808,e3bxDW,FEM,28.0,-,-,-,-,Graduado,...,False,True,False,False,True,True,False,False,False,0.020408
19707,79455,1112464706,mzdokNQ,MASC,22.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.099099
19590,79122,1112464628,2zkzEvx,FEM,20.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
73448,71633,1112462848,xkdO98x,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
81237,28807,1112366593,5l0VRZ,FEM,40.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.000000


In [33]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.94238344, 0.05761656],
       [0.89617899, 0.10382101],
       [0.90227165, 0.09772835],
       ...,
       [0.99256907, 0.00743093],
       [0.98846665, 0.01153335],
       [0.99798068, 0.00201932]])

In [34]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [35]:
prediccion['sepostulo']=y_final2

In [36]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.057617
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.103821
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.097728
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.009877
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.025875


In [37]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.057617
1,1,0.103821
2,2,0.097728
3,3,0.009877
4,4,0.025875


In [38]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion34.csv",index=False)

In [39]:
prediccion.sepostulo.mean()

0.5111304203260066